In [ ]:
# TR/opw00001
from pykiwoom.kiwoom import *

# 로그인
kiwoom = Kiwoom()
kiwoom.CommConnect(block=True)

# 계좌번호
account_list = kiwoom.GetLoginInfo("ACCNO")
account = account_list[0]
print(account)

print(kiwoom.GetConnectState())


In [ ]:
account

In [ ]:
# opw00001 요청
wallet_df = kiwoom.block_request("opw00001",
                          계좌번호=account,
                          비밀번호="",
                          비밀번호입력매체구분="00",
                          조회구분=2,
                          output="예수금상세현황",
                          next=0)

In [ ]:
def get_wallet():
    # opw00001 요청
    wallet_df = kiwoom.block_request("opw00001",
    계좌번호=account,
    비밀번호="",
    비밀번호입력매체구분="00",
    조회구분=2,
    output="예수금상세현황",
    next=0)

    return int(wallet_df['예수금'][0])

In [ ]:
# 예수금
예수금 = get_wallet()

In [ ]:
print(예수금)

키움 OpenAPI+의 TR을 사용하기 위해서는 Kiwoom 클래스의 block_request 메서드를 사용합니다. 첫 번째 인자로 TR의 이름을 적고 입력 값을 적어줍니다. 그리고 출력의 레코드 이름을 적어줍니다. 여기서 출력의 레코드는 위 그림에서 output 항목에 있는 '주식기본정보' 입니다. 마지막으로 단일 조회이면 next=0을 적어줍니다.

block_request 메서드의 출력은 DataFrame 객체입니다.

In [ ]:
test_df = kiwoom.block_request("opt10001",
                          종목코드="038500",
                          output="주식기본정보",
                          next=0)
test_df

In [20]:
kiwoom.GetMasterStockState("005930")

['증거금20%', '담보대출', '신용가능']

In [ ]:
name = kiwoom.GetMasterCodeName("038500")
print(name)
abs(int(test_df['현재가']))
# 시장가주문 매수
kiwoom.SendOrder("시장가매수", "0101", account, 1, "038500", 2, 0, "03", "")

In [ ]:
# 시장가주문 매도
kiwoom.SendOrder("시장가매도", "0101", account, 2, "038500", 2, 0, "03", "")

In [22]:
# 주식 이름
name = kiwoom.GetMasterCodeName("038500")
name

'삼표시멘트'

In [1]:
# 프로그램 코드 (키움 증권 연동)
from pykiwoom.kiwoom import *
import time

# 로그인
kiwoom = Kiwoom()
kiwoom.CommConnect(block=True)

ModuleNotFoundError: No module named 'pykiwoom'

In [12]:
kiwoom.CommConnect(block=True) # 지속적으로 연결이 안되는건가? 주식 거래 체결하고나면 다시 거래 안되고, 이것 다시 실행해서 제연결해주면 됨.

In [2]:
import time

In [2]:
# 계좌번호
account_list = kiwoom.GetLoginInfo("ACCNO")
account = account_list[0]

print(account)
print(kiwoom.GetConnectState())

# 프로그램 코드 (초기값 및 함수 정의)

class Stock():
    def __init__(self, stock_code):
        self.stock_code = stock_code # 주식 주문 코드
        self.stock_price = get_stock_price(self.stock_code) # 주식 현재가

        self.will_buy = False # 매수 희망 유무
        self.target_buy_price = 0 # 목표로 하는 매수가. 이것보다 현재가가 싸면 매수.
        self.buy_amount = 1 # 매수를 할 주식 개수
        self.maximum = 1000 # 최대 주식 갯수, 여기에 도달하면 더 이상 매수 안 함.
        self.buy_time = 60 # 
        self.last_buy_time = time.
        
        self.will_sell = False # 매도 희망 유무
        self.target_sell_price = 1000000000 # 목표로 하는 매도가. 이것보다 현재가가 비싸면 매도.
        self.sell_amount = 1 # 매도를 할 주식 개수
        self.sell_time = 60

    def modify_will_buy(self, modifed_amount):
        self.will_buy = modifed_amount

    def modify_will_sell(self, modifed_amount):
        self.will_sell = modifed_amount

    def buy_stock(self, wallet): # 주식 매수 함수
        stock_price = get_stock_price(self.stock_code)
        if((wallet >= stock_price*self.buy_amount) and (stock_price <= self.target_buy_price) and self.buy_time):
            # 시장가주문 매수
            kiwoom.SendOrder("시장가매수", "0101", account, 1, self.stock_code, self.buy_amount, 0, "03", "")
            self.last_buy_time = time

    def sell_stock(self, stock_amount): # 주식 매도 함수
        stock_price = get_stock_price(self.stock_code)
        # 시장가주문 매도
        if((stock_amount >= self.sell_amount) and (stock_price >= self.target_sell_price) and self.buy_time):
            kiwoom.SendOrder("시장가매도", "0101", account, 2, self.stock_code, self.sell_amount, 0, "03", "")

def get_wallet(): # 주문 가능 금액(예수금) 확인 함수
    # opw00001 요청
    wallet_df = kiwoom.block_request("opw00001",
        계좌번호=account,
        비밀번호="",
        비밀번호입력매체구분="00",
        조회구분=2,
        output="예수금상세현황",
        next=0
    )
    money = int(wallet_df['주문가능금액'][0])
    return money

def get_stock_price(stock_code): # 주식 현재가 확인 함수
    stock_df = kiwoom.block_request("opt10001",
        종목코드=stock_code,
        output="주식기본정보",
        next=0
    )
    print(stock_df['현재가']) # 에러 확인
    price = int(stock_df['현재가'])
    return price

def trade_function(stock): # 거래 결정 함수 (이걸 계속 돌리면 될 듯.)
    wallet = get_wallet()
    

5611439710
1


In [14]:
# 프로그램 코드 (테스트)
stock_code = "038500"

sampyo = Stock(stock_code)
sampyo.buy_stock(get_wallet())


{'trcode': 'opt10001', 'input': [{'주식기본정보요청': ['종목코드']}], 'output': [{'주식기본정보': ['종목코드', '종목명', '결산월', '액면가', '자본금', '상장주식', '신용비율', '연중최고', '연중최저', '시가총액', '시가총액비중', '외인소진률', '대용가', 'PER', 'EPS', 'ROE', 'PBR', 'EV', 'BPS', '매출액', '영업이익', '당기순이익', '250최고', '250최저', '시가', '고가', '저가', '상한가', '하한가', '기준가', '예상체결가', '예상체결수량', '250최고가일', '250최고가대비율', '250최저가일', '250최저가대비율', '현재가', '대비기호', '전일대비', '등락율', '거래량', '거래대비', '액면가단위', '유통주식', '유통비율']}]}
0    3595
Name: 현재가, dtype: object
{'trcode': 'opw00001', 'input': [{'예수금상세현황요청': ['계좌번호', '비밀번호', '비밀번호입력매체구분', '조회구분']}], 'output': [{'예수금상세현황': ['예수금', '주식증거금현금', '수익증권증거금현금', '익일수익증권매도정산대금', '해외주식원화대용설정금', '신용보증금현금', '신용담보금현금', '추가담보금현금', '기타증거금', '미수확보금', '공매도대금', '신용설정평가금', '수표입금액', '기타수표입금액', '신용담보재사용', '코넥스기본예탁금', 'ELW예탁평가금', '신용대주권리예정금액', '생계형가입금액', '생계형입금가능금액', '대용금평가금액(합계)', '잔고대용평가금액', '위탁대용잔고평가금액', '수익증권대용평가금액', '위탁증거금대용', '신용보증금대용', '신용담보금대용', '추가담보금대용', '권리대용금', '출금가능금액', '랩출금가능금액', '주문가능금액', '수익증권매수가능금액', '20%종목주문가능금액', '30%종목주문가능금액'

In [13]:
sampyo.sell_stock(get_wallet())

{'trcode': 'opw00001', 'input': [{'예수금상세현황요청': ['계좌번호', '비밀번호', '비밀번호입력매체구분', '조회구분']}], 'output': [{'예수금상세현황': ['예수금', '주식증거금현금', '수익증권증거금현금', '익일수익증권매도정산대금', '해외주식원화대용설정금', '신용보증금현금', '신용담보금현금', '추가담보금현금', '기타증거금', '미수확보금', '공매도대금', '신용설정평가금', '수표입금액', '기타수표입금액', '신용담보재사용', '코넥스기본예탁금', 'ELW예탁평가금', '신용대주권리예정금액', '생계형가입금액', '생계형입금가능금액', '대용금평가금액(합계)', '잔고대용평가금액', '위탁대용잔고평가금액', '수익증권대용평가금액', '위탁증거금대용', '신용보증금대용', '신용담보금대용', '추가담보금대용', '권리대용금', '출금가능금액', '랩출금가능금액', '주문가능금액', '수익증권매수가능금액', '20%종목주문가능금액', '30%종목주문가능금액', '40%종목주문가능금액', '100%종목주문가능금액', '현금미수금', '현금미수연체료', '현금미수금합계', '신용이자미납', '신용이자미납연체료', '신용이자미납합계', '기타대여금', '기타대여금연체료', '기타대여금합계', '미상환융자금', '융자금합계', '대주금합계', '신용담보비율', '중도이용료', '최소주문가능금액', '대출총평가금액', '예탁담보대출잔고', '매도담보대출잔고', 'd+1추정예수금', 'd+1매도매수정산금', 'd+1매수정산금', 'd+1미수변제소요금', 'd+1매도정산금', 'd+1출금가능금액', 'd+2추정예수금', 'd+2매도매수정산금', 'd+2매수정산금', 'd+2미수변제소요금', 'd+2매도정산금', 'd+2출금가능금액', '출력건수']}, {'종목별예수금현황': ['통화코드', '외화예수금', '원화대용평가금', '해외주식증거금', '출금가능금액(예수금)', '주문가능금액(예수금)', '외화미수(합계